<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Импорт библиотек

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
#from scipy import stats as st

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [2]:
import catboost
import time

from tqdm import tqdm
# from keras.models import Model
# from keras.layers import LSTM, Dense, Masking, Input, Concatenate, Reshape
# from keras.metrics import AUC
# from keras.utils import pad_sequences, plot_model, set_random_seed
# from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.regularizers import l1,l2
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from catboost import CatBoostClassifier, Pool, cv

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Загрузка файла с данными
total_scores = pd.read_csv('data_01_predict_progress/total_scores.csv')
tournament_scores = pd.read_csv('data_01_predict_progress/tournament_scores.csv')
tournaments = pd.read_csv('data_01_predict_progress/tournaments.csv')
units = pd.read_csv('data_01_predict_progress/units.csv')


In [7]:
units

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0
...,...,...,...
4591,10122,green,198.0
4592,5108,green,62.0
4593,5109,green,27.0
4594,5110,green,62.0


In [25]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB


In [24]:
units['color'].unique()

array(['green', 'lime', nan], dtype=object)

In [26]:
units['color'].value_counts()

color
green    3800
lime      795
Name: count, dtype: int64

In [29]:
units['school_id'].value_counts()

school_id
62.0     348
198.0    345
111.0    255
38.0     241
192.0    124
        ... 
2.0        1
267.0      1
163.0      1
191.0      1
227.0      1
Name: count, Length: 239, dtype: int64

units
Юнит – спортивная единица. Мы работаем с одиночным катанием, в одном юните только один спортсмен. Если бы были пары/команды, то было б несколько спортсменов в юните.
- id: идентификатор юнита
- color: категория
- school_id: идентификатор школы

In [8]:
tournaments

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0
...,...,...,...,...
137,7114,2092-04-15,2092-04-20,1.0
138,7116,2091-10-24,2091-10-27,0.0
139,7117,2092-04-29,2092-05-03,0.0
140,7115,2092-04-23,2092-04-26,2.0


tournaments
Турнир состоит из нескольких категорий, оценки по категориям расписаны в total_scores
- id: идентификатор турнира
- date_start: дата начала
- date_end: дата завершения
- origin_id: место проведения


In [6]:
total_scores

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,6,20,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,20,71.66,20
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,7,21,Произвольная программа,q Прыжок приземлён в четверть e Явно неправиль...,21,71.27,21
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,5,22,Произвольная программа,q Прыжок приземлён в четверть << Пониженный пр...,22,70.49,22
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,2,23,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,24,68.22,24


Оценки за выступления по категориям и общие за турнир
- id: идентификатор выступления, джойнится с tournament_scores.total_score_id
- unit_id: идентификатор юнита, ключ к units.id
- tournament_id: идентификатор турнира, tournaments.id
- components_score: артистизм (мастерство, композиция, хореография)
- base_score: базовая оценка за элементы в выступлении (идеал)
- elements_score: реальная оценка всех выполненных элементов, base_score+goe
- decreasings_score: снижения оценок за ошибки
- total_score: components_score+elements_score+decreasings_score за выступление
- starting_place:
- place: занятое место в категории category_name+segment_name
- segment_name: название сегмента
- info: комментарии и пояснения к оценке
- overall_place: итоговое место в турнире
- overall_total_score: итоговая оценка за весь турнир
- overall_place_str: комментарии, пояснения

In [19]:
display(tournament_scores)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09
2,3,1,3Lz,NaN,0.0,-1.30,5.19
3,4,1,CCoSp4,NaN,3.5,0.56,4.06
4,5,1,FCSp4,NaN,3.2,0.64,3.84
...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12


In [33]:
tournament_scores['decrease'].unique()

array([nan, 'q', '<', '!', '<<', 'nS', 'F', '*', 'e', 'nU', 'nC', '<<*',
       '<*', 'B', 'nS*', '!F', 'F*', '!*', 'q*', 'e*', 'nF', 'nB', 'B.',
       'f*', 'В', '<F', 'qF', 'b', '!<', 'f', '!F*', 'FnU', 'FF'],
      dtype=object)

- '*' - элемент не засчитывается
- q - недокручена четверть
- < - недокручено 90-180 градусов
- << - недокручено больше 180 градусов
- e  - отталкивание не с того ребра
- ! - толчок не с точного ребра
- F - упал
- B - bonus 
- nU
- nC - не было либелы
- nS - не было волчка
- nF
- nB
- v -  можно заменить все ошибки во вращениях

In [ ]:
tournament_scores['title'].nunique()

tournament_scores Таблица с оценками поэлементно
- id: идентификатор оценки за конкретный элемент/комбинацию
- total_score_id: идентикатор выступления, ключ total_scores.id
- title: запись элемента или комбинации элементов с отметками об ошибках
- decrease: за что снижена оценка
- base_score: базовая оценка (идеал, цена данного элемента/комбинации, сложность)
- goe: Grade of Execute, качество исполнения, судейские надбавки/убавки
- avg_score: оценка за элемент/комбинацию (усредненная по судьям)